# ANNUITY CALCULATOR

In this notebook I genarate annuity cash flows.

In [1]:
from financepy.utils import *

####################################################################
# FINANCEPY BETA Version 0.260 - This build:  22 Nov 2022 at 12:35 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



In [2]:
from financepy.products.bonds import *
from financepy.utils import *
from financepy.products.rates import *

## CREATE AN ANNUITY STREAM

Calculate the cash flows on an annuity

In [3]:
valuation_date = Date(20, 6, 2018)
settlement_date = valuation_date

In [4]:
maturity_date = settlement_date.add_tenor("5Y")  # Date(20, 9, 2028)
coupon = 0.05
frequencyType = FrequencyTypes.SEMI_ANNUAL
calendar_type = CalendarTypes.WEEKEND
bus_day_adjust_type = BusDayAdjustTypes.FOLLOWING
date_gen_rule_type = DateGenRuleTypes.BACKWARD
basis_type = DayCountTypes.ACT_360
face = ONE_MILLION

In [5]:
annuity = BondAnnuity(maturity_date,coupon,frequencyType,calendar_type,
                         bus_day_adjust_type,date_gen_rule_type,basis_type,face)

In [6]:
print(annuity)

OBJECT TYPE: BondAnnuity
MATURITY DATE: 20-JUN-2023
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
CALENDAR: CalendarTypes.WEEKEND
BUS_DAY_RULE: BusDayAdjustTypes.FOLLOWING
DATE_GEN_RULE: DateGenRuleTypes.BACKWARD



In [7]:
annuity.print_flows(settlement_date)

20-DEC-2018 , 25416.666666666668
20-JUN-2019 , 25277.777777777777
20-DEC-2019 , 25416.666666666668
22-JUN-2020 , 25694.44444444444
21-DEC-2020 , 25277.777777777777
21-JUN-2021 , 25277.777777777777
20-DEC-2021 , 25277.777777777777
20-JUN-2022 , 25277.777777777777
20-DEC-2022 , 25416.666666666668
20-JUN-2023 , 25277.777777777777


## We can value the flows by discounting them

First we build a Libor swap curve

In [8]:
depos = []
dcType = DayCountTypes.ACT_360
fixedFreq = FrequencyTypes.SEMI_ANNUAL
swapType = SwapTypes.PAY
swap1 = IborSwap(settlement_date,"1Y",swapType,0.0500,fixedFreq,dcType)
swap2 = IborSwap(settlement_date,"2Y",swapType,0.0500,fixedFreq,dcType)
swap3 = IborSwap(settlement_date,"3Y",swapType,0.0500,fixedFreq,dcType)
swap4 = IborSwap(settlement_date,"4Y",swapType,0.0500,fixedFreq,dcType)
swap5 = IborSwap(settlement_date,"5Y",swapType,0.0500,fixedFreq,dcType)
swaps = [swap1, swap2, swap3, swap4, swap5]

libor_curve = IborSingleCurve(valuation_date, depos, [], swaps)

In [9]:
swap5.value(settlement_date, libor_curve)

5.820766091346741e-11

In [10]:
swap5.print_fixed_leg_pv()

START DATE: 20-JUN-2018
MATURITY DATE: 20-JUN-2023
COUPON (%): 5.0
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.ACT_360
PAY_DATE     ACCR_START   ACCR_END     DAYS  YEARFRAC    RATE      PAYMENT       DF          PV        CUM PV
20-DEC-2018  20-JUN-2018  20-DEC-2018   183  0.508333   5.00000     25416.67  0.97521251     24786.65     24786.65
20-JUN-2019  20-DEC-2018  20-JUN-2019   182  0.505556   5.00000     25277.78  0.95116989     24043.46     48830.11
20-DEC-2019  20-JUN-2019  20-DEC-2019   183  0.508333   5.00000     25416.67  0.92756556     23575.62     72405.74
22-JUN-2020  20-DEC-2019  22-JUN-2020   185  0.513889   5.00000     25694.44  0.90435730     23236.96     95642.70
21-DEC-2020  22-JUN-2020  21-DEC-2020   182  0.505556   5.00000     25277.78  0.88208873     22297.24    117939.94
21-JUN-2021  21-DEC-2020  21-JUN-2021   182  0.505556   5.00000     25277.78  0.86031325     21746.81    139686.75
20-DEC-2021  21-JUN-2021  20-DEC-2021   182  0.505556   5.0000

Here are the flows on the fixed leg of the 5Y swap

In [11]:
swap5.print_float_leg_pv()

START DATE: 20-JUN-2018
MATURITY DATE: 20-JUN-2023
SPREAD (BPS): 0.0
FREQUENCY: FrequencyTypes.QUARTERLY
DAY COUNT: DayCountTypes.THIRTY_E_360
PAY_DATE     ACCR_START   ACCR_END     DAYS  YEARFRAC    IBOR      PAYMENT       DF          PV        CUM PV
20-SEP-2018  20-JUN-2018  20-SEP-2018    90  0.250000    5.07939     12698.47  0.98746076     12539.24     12539.24
20-DEC-2018  20-SEP-2018  20-DEC-2018    90  0.250000    5.02383     12559.58  0.97521251     12248.26     24787.49
20-MAR-2019  20-DEC-2018  20-MAR-2019    90  0.250000    4.96828     12420.70  0.96324829     11964.22     36751.71
20-JUN-2019  20-MAR-2019  20-JUN-2019    90  0.250000    5.07939     12698.47  0.95116989     12078.40     48830.11
20-SEP-2019  20-JUN-2019  20-SEP-2019    90  0.250000    5.08536     12713.40  0.93922909     11940.80     60770.91
20-DEC-2019  20-SEP-2019  20-DEC-2019    90  0.250000    5.02974     12574.35  0.92756556     11663.53     72434.44
20-MAR-2020  20-DEC-2019  20-MAR-2020    90  0.2500

In [12]:
cleanValue = annuity.clean_price_from_discount_curve(settlement_date, libor_curve)

In [13]:
dirtyValue = annuity.dirty_price_from_discount_curve(settlement_date, libor_curve)

The price of the annuity is quoted based on a par notional

In [14]:
print("CLEAN PRICE:", cleanValue)
print("DIRTY PRICE:", dirtyValue)
print("ACCRD VALUE:", annuity._accrued_interest)

CLEAN PRICE: 22.154839385148083
DIRTY PRICE: 22.154839385148083
ACCRD VALUE: 0.0


Copyright (c) 2020 Dominic O'Kane